In [1]:
###############################################
## Import the core Scopus Dataset (previusly mined) --- use Publication EID as unique key to grab co-authorship network data
##
## Note: Limitted to 10k API calls per week; so I will run in two parts (part 1 = eid[1:10000]; part 2 = eid[10001:end])
##
## Author: Christopher Meaney
## Date: January 2023
##############################################

In [2]:
########################
## Dependency packages
########################
library(rscopus)

In [3]:
#########################
## Set Scopus API key
#########################
api_key <- set_api_key("Enter_Your_API_Key_Here")

In [4]:
#########################
## Filepaths where data will be imported from, or exported to
#########################

## Set working directory path
wd_path <- "Enter_a_Path_to_a_Working_Directory_For_This_Project"

## Path to core mined scopus data
scopus_fpath <- paste0(wd_path, "//Core_Scopus_Dataset_Sm.csv")

## Path to where eid data will live
auths_eid_fpath <- paste0(wd_path, "//Scopus_CoAuthNetwork_EID1.csv")

## Path to where affiliation data will live
affils_eid_fpath <- paste0(wd_path, "//Scopus_CoAuthAffiliationNetwork_EID1.csv")

In [5]:
#####################
## Import core Scopus Dataset
#####################
scopus_data <- read.csv(scopus_fpath, header=TRUE, sep=",", stringsAsFactors=FALSE)
str(scopus_data)

'data.frame':	18874 obs. of  25 variables:
 $ au_id         : num  1e+10 1e+10 1e+10 1e+10 1e+10 ...
 $ prism_url     : chr  "https://api.elsevier.com/content/abstract/scopus_id/85140976962" "https://api.elsevier.com/content/abstract/scopus_id/85121747496" "https://api.elsevier.com/content/abstract/scopus_id/85112107056" "https://api.elsevier.com/content/abstract/scopus_id/85118672412" ...
 $ eid           : chr  "2-s2.0-85140976962" "2-s2.0-85121747496" "2-s2.0-85112107056" "2-s2.0-85118672412" ...
 $ doi           : chr  "10.1016/j.socscimed.2022.115463" "10.1111/add.15760" "10.1016/j.socscimed.2021.114262" "10.3399/BJGP.2021.0195" ...
 $ issn          : chr  "02779536" "09652140" "02779536" "09601643" ...
 $ eissn         : chr  "18735347" "13600443" "18735347" "14785242" ...
 $ pub_title     : chr  "Brief opportunistic interventions by general practitioners to promote smoking cessation: A conversation analytic study" "The old and familiar meets the new and unknown: patient and clin

In [6]:
table(scopus_data$au_institution, useNA="always")


Cambridge Dartmouth      Duke   Harvard       HKU     Laval    McGill  Michigan 
      560      1173       380       485       648       785      1025       793 
   Monash      OHSU    Ottawa    Oxford   Toronto       UBC       UCL      UCSF 
      455       752       774      5443      2484       920      1526       421 
     UNSW      <NA> 
      250         0 

In [7]:
sum(table(scopus_data$au_institution, useNA="always"))

[1] 18874

In [8]:
##
##
## Get list of unique eid values corresponding to primary care research publications
##
##

In [9]:
## Grab eid's for mining
eid_vec <- scopus_data$eid
length(eid_vec)

[1] 18874

In [10]:
eid_vec <- eid_vec[!(is.na(eid_vec))]
length(eid_vec)

[1] 18874

In [11]:
uniq_eid <- unique(eid_vec)
length(uniq_eid)

[1] 13098

In [12]:
##
## THIS IS A HACK WHERE I GRAB SAY 9k eids; then leave the rest for next week (part 2; since 10k API queries per week)
##
uniq_eid <- uniq_eid[1:9000]
length(uniq_eid)

[1] 9000

In [13]:
##################################
##
##
## Get additional article information - querying by "eid"
##
##
##################################
get_auths_affils <- function(eid) {
        out <- tryCatch(
        {
        ## Grab data for given eid
        dat <- abstract_retrieval(id=eid, identifier = c("eid"))
        ## For each eid dataframe grab relevant info
        num_auth <- length(dat[["content"]][["abstracts-retrieval-response"]][["authors"]][[1]])
        ## Grab co-author names
        auths <- list()
        ids <- list()
        for (i in 1:num_auth) {
            auths[[i]] <- dat[["content"]][["abstracts-retrieval-response"]][["authors"]][[1]][[i]][["ce:indexed-name"]]
            ids[[i]] <- dat[["content"]][["abstracts-retrieval-response"]][["authors"]][[1]][[i]][["@auid"]]	
        }
        auths_vec <- unlist(auths)
        ids_vec <- unlist(ids)
        ## Put together into small author dataframe
        dat_sm_auths <- data.frame(eid=rep(eid, num_auth),
                        auths=auths_vec,
                        ids=ids_vec)
        ## Grab affiliation names
        affil_city <- list()
        affil_id <- list()
        affil_name <- list()
        affil_country <- list()
        if (is.list(dat[["content"]][["abstracts-retrieval-response"]][["affiliation"]][[1]])==TRUE) {
            num_affil <- length(dat[["content"]][["abstracts-retrieval-response"]][["affiliation"]])
            for (i in 1:num_affil) {
                affil_city[[i]] <- dat[["content"]][["abstracts-retrieval-response"]][["affiliation"]][[i]][["affiliation-city"]]
                affil_id[[i]] <- dat[["content"]][["abstracts-retrieval-response"]][["affiliation"]][[i]][["@id"]]
                affil_name[[i]] <- dat[["content"]][["abstracts-retrieval-response"]][["affiliation"]][[i]][["affilname"]]
                affil_country[[i]] <- dat[["content"]][["abstracts-retrieval-response"]][["affiliation"]][[i]][["affiliation-country"]]
            }
        } else {
            num_affil <- 1
            for (i in 1:num_affil) {
                affil_city[[i]] <- dat[["content"]][["abstracts-retrieval-response"]][["affiliation"]][["affiliation-city"]]
                affil_id[[i]] <- dat[["content"]][["abstracts-retrieval-response"]][["affiliation"]][["@id"]]
                affil_name[[i]] <- dat[["content"]][["abstracts-retrieval-response"]][["affiliation"]][["affilname"]]
                affil_country[[i]] <- dat[["content"]][["abstracts-retrieval-response"]][["affiliation"]][["affiliation-country"]]
            }
        }
        affil_city_vec <- unlist(affil_city)
        affil_id_vec <- unlist(affil_id)
        affil_name_vec <- unlist(affil_name)
        affil_country_vec <- unlist(affil_country)		
        ## Put together into small affiliation dataframe
        dat_sm_affil <- data.frame(eid=rep(eid, num_affil),
                        affil_city=affil_city_vec,
                        affil_id=affil_id_vec,
                        affil_name=affil_name_vec,
                        affil_country=affil_country_vec)
        ## Combine both author/affil information into single vector
        dat_sm_out <- list(dat_sm_auths=dat_sm_auths,
                        dat_sm_affil=dat_sm_affil)
        },
        error=function(cond) {
            # Choose a return value in case of error
            return(list(dat_sm_auths=data.frame(eid=eid,
                                                auths=NA,
                                                ids=NA),
                        dat_sm_affil=data.frame(eid=eid,
                                                affil_city=NA,
                                                affil_id=NA,
                                                affil_name=NA,
                                                affil_country=NA)))
        },
        warning=function(cond) {
            # Choose a return value in case of warning
            return(list(dat_sm_auths=data.frame(eid=eid,
                                                auths=NA,
                                                ids=NA),
                        dat_sm_affil=data.frame(eid=eid,
                                                affil_city=NA,
                                                affil_id=NA,
                                                affil_name=NA,
                                                affil_country=NA)))
        },
        finally={
            # Choose a return value in case of warning
        }
    )    
    return(out)
}


In [14]:
## Instantitate list to hold eid  based author information
auths_eid_list <- list()
affils_eid_list <- list()

##
## Loop over list grabbing data
##
t0 <- Sys.time()

for (i in 1:length(uniq_eid)) {
	## Grab Name and ID
	eid <- uniq_eid[i]
	cat(i, "\tQuerying Scopus EID: ", eid, "\n")
	## Add to list
	dat_list <- get_auths_affils(eid=uniq_eid[i])
	dat_sm_auths <- dat_list[[1]]
	dat_sm_affils <- dat_list[[2]]
	dat_sm_auths$doc_id <- i
	dat_sm_affils$doc_id <- i
	auths_eid_list[[i]] <- dat_sm_auths 
	affils_eid_list[[i]] <- dat_sm_affils
	## Pause for a second as to not kill API
	Sys.sleep(runif(n=1, min=0, max=2))
}

t1 <- Sys.time()
t1-t0

1 	Querying Scopus EID:  2-s2.0-85140976962 


HTTP specified is:https://api.elsevier.com/content/abstract/eid/2-s2.0-85140976962




2 	Querying Scopus EID:  2-s2.0-85121747496 


HTTP specified is:https://api.elsevier.com/content/abstract/eid/2-s2.0-85121747496




3 	Querying Scopus EID:  2-s2.0-85112107056 


...

8999 	Querying Scopus EID:  2-s2.0-85118834097 


HTTP specified is:https://api.elsevier.com/content/abstract/eid/2-s2.0-85118834097




9000 	Querying Scopus EID:  2-s2.0-85110447235 


HTTP specified is:https://api.elsevier.com/content/abstract/eid/2-s2.0-85110447235




Time difference of 3.433373 hours

In [15]:
## 
## Post Process the Data
##

In [16]:
auths_eid_df <- do.call("rbind", auths_eid_list)
str(auths_eid_df)

'data.frame':	102880 obs. of  4 variables:
 $ eid   : chr  "2-s2.0-85140976962" "2-s2.0-85140976962" "2-s2.0-85140976962" "2-s2.0-85140976962" ...
 $ auths : chr  "Wheat H." "Barnes R.K." "Aveyard P." "Stevenson F." ...
 $ ids   : chr  "56443889300" "10041373600" "55073731400" "7102932453" ...
 $ doc_id: int  1 1 1 1 1 2 2 2 2 2 ...


In [17]:
head(auths_eid_df)

,eid,auths,ids,doc_id
,<chr>,<chr>,<chr>,<int>
1,2-s2.0-85140976962,Wheat H.,56443889300,1
2,2-s2.0-85140976962,Barnes R.K.,10041373600,1
3,2-s2.0-85140976962,Aveyard P.,55073731400,1
4,2-s2.0-85140976962,Stevenson F.,7102932453,1
5,2-s2.0-85140976962,Begh R.,35097292400,1
6,2-s2.0-85121747496,Albury C.,57202071244,2


In [18]:
tail(auths_eid_df)

,eid,auths,ids,doc_id
,<chr>,<chr>,<chr>,<int>
102875,2-s2.0-85110447235,Bozzetti F.,35463734000,9000
102876,2-s2.0-85110447235,Strasser F.,7006755372,9000
102877,2-s2.0-85110447235,Thoresen L.,6601984319,9000
102878,2-s2.0-85110447235,Kazemi S.,57232134300,9000
102879,2-s2.0-85110447235,Baracos V.,7006668487,9000
102880,2-s2.0-85110447235,Senesse P.,6603768537,9000


In [19]:
affils_eid_df <- do.call("rbind", affils_eid_list)
str(affils_eid_df)

'data.frame':	43634 obs. of  6 variables:
 $ eid          : chr  "2-s2.0-85140976962" "2-s2.0-85140976962" "2-s2.0-85140976962" "2-s2.0-85121747496" ...
 $ affil_city   : chr  "Plymouth" "London" "Oxford" "London" ...
 $ affil_id     : chr  "60024779" "60022148" "60002634" "60022148" ...
 $ affil_name   : chr  "University of Plymouth" "University College London" "University of Oxford Medical Sciences Division" "University College London" ...
 $ affil_country: chr  "United Kingdom" "United Kingdom" "United Kingdom" "United Kingdom" ...
 $ doc_id       : int  1 1 1 2 2 2 2 3 3 3 ...


In [20]:
head(affils_eid_df)

,eid,affil_city,affil_id,affil_name,affil_country,doc_id
,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
1,2-s2.0-85140976962,Plymouth,60024779,University of Plymouth,United Kingdom,1
2,2-s2.0-85140976962,London,60022148,University College London,United Kingdom,1
3,2-s2.0-85140976962,Oxford,60002634,University of Oxford Medical Sciences Division,United Kingdom,1
4,2-s2.0-85121747496,London,60022148,University College London,United Kingdom,2
5,2-s2.0-85121747496,Nottingham,60015138,University of Nottingham,United Kingdom,2
6,2-s2.0-85121747496,Norwich,60011775,"University of East Anglia, Faculty of Medicine and Health Sciences",United Kingdom,2


In [21]:
tail(affils_eid_df)

,eid,affil_city,affil_id,affil_name,affil_country,doc_id
,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
43629,2-s2.0-85110447235,Edmonton,60019490,"University of Alberta, Faculty of Medicine and Dentistry",Canada,9000
43630,2-s2.0-85110447235,Montreal,60018916,Sir Mortimer B. Davis Jewish General Hospital,Canada,9000
43631,2-s2.0-85110447235,Toronto,60016849,University of Toronto,Canada,9000
43632,2-s2.0-85110447235,Montpellier,60013385,CRLC Centre Val d'Aurelle - Paul Lamarque,France,9000
43633,2-s2.0-85110447235,Chicago,60001681,Abbott Laboratories,United States,9000
43634,2-s2.0-85110447235,Thessaly,122420365,Animus-Kyanous Stavros Private Clinic,Greece,9000


In [22]:
#######################
## Write to disk
#######################

In [23]:
write.csv(x=auths_eid_df, file=auths_eid_fpath, row.names=FALSE)

In [24]:
write.csv(x=affils_eid_df, file=affils_eid_fpath, row.names=FALSE)

In [25]:
##########################
## Session Information
##########################

In [26]:
Sys.Date()

[1] "2023-01-23"

In [27]:
sessionInfo()

R version 4.1.3 (2022-03-10)
Platform: x86_64-w64-mingw32/x64 (64-bit)
Running under: Windows 10 x64 (build 17763)

Matrix products: default

locale:
[1] LC_COLLATE=English_United States.1252 
[2] LC_CTYPE=English_United States.1252   
[3] LC_MONETARY=English_United States.1252
[4] LC_NUMERIC=C                          
[5] LC_TIME=English_United States.1252    

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] rscopus_0.7.1

loaded via a namespace (and not attached):
 [1] Rcpp_1.0.8.3     magrittr_2.0.3   tidyselect_1.1.2 uuid_1.1-0      
 [5] R6_2.5.1         rlang_1.0.2      fastmap_1.1.0    fansi_1.0.3     
 [9] plyr_1.8.7       httr_1.4.4       dplyr_1.0.9      tools_4.1.3     
[13] utf8_1.2.2       cli_3.3.0        htmltools_0.5.2  ellipsis_0.3.2  
[17] digest_0.6.29    tibble_3.1.7     lifecycle_1.0.1  crayon_1.5.1    
[21] IRdisplay_1.1    tidyr_1.2.0      purrr_0.3.4      repr_1.1.4      
[25] base6

In [28]:
version

               _                           
platform       x86_64-w64-mingw32          
arch           x86_64                      
os             mingw32                     
system         x86_64, mingw32             
status                                     
major          4                           
minor          1.3                         
year           2022                        
month          03                          
day            10                          
svn rev        81868                       
language       R                           
version.string R version 4.1.3 (2022-03-10)
nickname       One Push-Up                 